In [ ]:
!pip install openpyxl

In [ ]:
import os
import yaml
import glob
import io
import pandas as pd
from lib.utils import process_document
from rdflib import plugin, ConjunctiveGraph, URIRef
from rdflib.store import Store
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer

pd.set_option('display.max_colwidth', 200)

if not os.path.exists('outputs'):
    os.makedirs('outputs')

In [ ]:
vocab = dict()
vocab[None] = URIRef('http://envri.eu/ns/NULL')
vocab['NULL'] = URIRef('http://envri.eu/ns/NULL')

store = plugin.get('IOMemory', Store)()

g = ConjunctiveGraph(store)
g.bind('envri', 'http://envri.eu/ns/')
g.bind('dcterms', 'http://purl.org/dc/terms/')
g.bind('foaf', 'http://xmlns.com/foaf/0.1/')
g.bind('skos', 'http://www.w3.org/2004/02/skos/core#')

with open('vocab.yaml', 'r') as f:
    for key, value in yaml.safe_load(f).items():
        vocab[key] = URIRef(value)

with open('fairmapping.yaml', 'r') as f:
    for key, value in yaml.safe_load(f).items():
        g.add((vocab[key], vocab['relatesTo'], URIRef(value)))
    
for file in glob.glob('descriptions/*.yaml'):
    with open(file, 'r') as f:
        for document in yaml.load_all(f, Loader=yaml.FullLoader):
            process_document(store, document, vocab)
    
g.serialize(destination='data.trig', format='trig')

In [ ]:
g = ConjunctiveGraph()
g.parse('data.trig', format='trig')

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = io.BytesIO()
    serializer.serialize(output)
    return pd.read_csv(io.StringIO(output.getvalue().decode('utf-8')), encoding='utf-8')
    
def write(df, fn):
    df.to_excel('outputs/{}'.format(fn), encoding='utf-8')

In [ ]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?ri_acronym ?ri_url ?ri_domain ?rep_label WHERE {
    ?g dcterms:date ?date .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri envri:acronym ?ri_acronym . 
        ?ri envri:riUrl ?ri_url .
        ?ri envri:hasDomain ?ri_domain .
        ?ri envri:hasRepository ?rep .
        ?rep a envri:Repository .
        ?rep rdfs:label ?rep_label .
    }
    FILTER (?date > "2019-01-01"^^xsd:date)
    # FILTER (?ri_acronym = "ICOS")
}
""")

display(q)
write(q, 'output-1.xlsx')

In [ ]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?rep_label ?vocab_name ?vocab_iri WHERE {
    [] a rm:ResearchInfrastructure ;
       envri:hasDomain envri:Earth ;
       envri:hasRepository [
         a envri:Repository ;
         rdfs:label ?rep_label ;
         envri:hasVocabularies [
           envri:hasName ?vocab_name ;
           envri:hasVocabularyIri ?vocab_iri 
         ]
       ]
}
""")

display(q)
write(q, 'output-2.xlsx')

In [ ]:
# All properties that relate to a specific FAIR principle
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>

SELECT ?p WHERE {
    ?p envri:relatesTo fairterms:A1.2 .
}
""")

display(q)
write(q, 'output-3.xlsx')

In [ ]:
# Retrieve the context of a property relating to a specific FAIR principle and filter those properties for which the value is NULL
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-4.xlsx')

In [ ]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?rep_label ?schema_name WHERE {
    [] a rm:ResearchInfrastructure ;
       envri:hasDomain envri:Earth ;
       envri:hasRepository [
         a envri:Repository ;
         rdfs:label ?rep_label ;
         envri:hasMetadata [
           envri:hasSchema [
             envri:hasSchemaName ?schema_name
           ]
         ]
       ]
}
""")

display(q)
write(q, 'output-5.xlsx')